In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
#pip install yfinance
#pip install scikit-learn
#pip install tweepy
#pip install mariadb

In [2]:
import yfinance as yf
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")



Mounted at /content/drive/


In [63]:
enddate = dt.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
startdate = dt.datetime.strptime("2022-02-01", "%Y-%m-%d").date()

data = yf.download(tickers='ICLN', start=startdate, end=enddate, interval='1d',prepost= True)

[*********************100%***********************]  1 of 1 completed


In [64]:
def bollinger_band(data):
    window = 20
    rolling_mean = data['Close'].rolling(window).mean()
    rolling_std = data['Close'].rolling(window).std()
    
    data['Middle Band'] = rolling_mean
    data['Short Upper Band'] = data['Middle Band'] + 1.5*rolling_std  # Calculates 10 day moving average for bands
    data['Short Lower Band'] = data['Middle Band'] - 1.5*rolling_std
    
    data['Medium Upper Band'] = data['Middle Band'] + 2*rolling_std  # Calculates 20 day moving average for bands
    data['Medium Lower Band'] = data['Middle Band'] - 2*rolling_std
    
    data['Long Upper Band'] = data['Middle Band'] + 2.5*rolling_std  # Calculates 50 day moving average for bands
    data['Long Lower Band'] = data['Middle Band'] - 2.5*rolling_std
    
    
bollinger_band(data)



In [65]:
"""
RSI and Bollinger Band:
https://www.analyticsvidhya.com/blog/2021/01/algotrading-using-technical-indicator-and-ml-models/
"""

def rsi(data, window = 14):
    close = data["Close"]
    
    #Price increase or decrease over previous day
    dif = close.diff()
    dif = dif[1:]
    
    # pos_m identifies stock price going up
    # neg_m identifies stock price going down
    pos_m, neg_m = dif.copy(), dif.copy()
    pos_m[pos_m < 0] = 0
    neg_m[neg_m > 0] = 0
    
    # Positive Rolling Mean Exponential
    prme = pos_m.ewm(span=window).mean()
    # Negative Rolling mean Exponential 
    nrme = neg_m.abs().ewm(span=window).mean()
    
    # Ratio of magnitude of up move to down move
    RSE = prme / nrme
    RSIE = 100 - (100/(1+RSE))
    data["RSIE"] = RSIE
    
    # Positive Rolling Mean Simple
    prms = pos_m.rolling(window).mean()
    # Negative Rolling Mean Simple
    nrms = neg_m.rolling(window).mean()
    
    RSS = prms / nrms
    RSIS = 100.0 - (100.0 / (1.0 + RSS))
    
    data['rsis'] = RSIS
    return data

rsi(data)

,Open,High,Low,Close,Adj Close,Volume,Middle Band,Short Upper Band,Short Lower Band,Medium Upper Band,Medium Lower Band,Long Upper Band,Long Lower Band,RSIE,rsis
Date,,,,,,,,,,,,,,,
2022-02-01,18.799999,18.799999,18.400000,18.719999,18.560610,5726200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-02,18.910000,18.920000,18.570000,18.639999,18.481293,5861900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
2022-02-03,18.379999,18.520000,18.059999,18.170000,18.015295,6870200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
2022-02-04,18.010000,18.500000,17.969999,18.340000,18.183847,5079300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.669957,NaN
2022-02-07,18.240000,18.469999,18.150000,18.230000,18.074785,4423000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.241263,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-25,19.570000,19.760000,19.330000,19.750000,19.715769,2733100,19.3300,19.847331,18.812668,20.019775,18.640225,20.192219,18.467781,60.614587,314.084848
2022-07-26,19.610001,19.680000,19.410000,19.440001,19.406307,2764400,19.3180,19.822199,18.813800,19.990266,18.645734,20.158332,18.477667,50.673982,464.581835
2022-07-27,19.889999,20.389999,19.660000,20.360001,20.324711,5195900,19.3785,19.987420,18.769580,20.190393,18.566607,20.393366,18.363634,68.412720,382.257375


In [66]:
#declare figure
fig = go.Figure()



fig.add_trace(go.Scatter(x=data.index, y= data['Middle Band'],line=dict(color='blue', width=.7), name = 'Middle Band'))

fig.add_trace(go.Scatter(x=data.index, y= data['Short Upper Band'],line=dict(color='pink', width=1.5), name = 'Short Upper Band (Sell)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Short Lower Band'],line=dict(color='lightgreen', width=1.5), name = 'Short Lower Band (Buy)'))

fig.add_trace(go.Scatter(x=data.index, y= data['Medium Upper Band'],line=dict(color='red', width=1.5), name = 'Medium Upper Band (Sell)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Medium Lower Band'],line=dict(color='green', width=1.5), name = 'Medium Lower Band (Buy)'))

fig.add_trace(go.Scatter(x=data.index, y= data['Long Upper Band'],line=dict(color='darkred', width=1.5), name = 'Long Upper Band (Sell)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Long Lower Band'],line=dict(color='darkgreen', width=1.5), name = 'Long Lower Band (Buy)'))
    
    



In [67]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y= data['Middle Band'],line=dict(color='blue', width=.7), name = 'Middle Band'))

fig.add_trace(go.Scatter(x=data.index, y= data['Short Upper Band'],line=dict(color='pink', width=1.5), name = 'Short Upper Band (Sell)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Medium Upper Band'],line=dict(color='red', width=1.5), name = 'Medium Upper Band (Sell)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Long Upper Band'],line=dict(color='darkred', width=1.5), name = 'Long Upper Band (Sell)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Short Lower Band'],line=dict(color='lightgreen', width=1.5), name = 'Short Lower Band (Buy)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Medium Lower Band'],line=dict(color='green', width=1.5), name = 'Medium Lower Band (Buy)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Long Lower Band'],line=dict(color='darkgreen', width=1.5), name = 'Long Lower Band (Buy)'))

#Candlestick
fig.add_trace(go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'], name = 'market data'))

# Add titles
fig.update_layout(
    title='ICLN live share price evolution',
    yaxis_title='Stock Price (USD per Shares)')

# X-Axes
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15m", step="minute", stepmode="backward"),
            dict(count=45, label="45m", step="minute", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(count=3, label="3h", step="hour", stepmode="backward"),
            dict(step="all")
        ])
    )
)



In [69]:
data.dropna(inplace=True)
data["BB_Signal"] = 0

data["BB_Signal"] = np.where(data["Low"] < data["Medium Lower Band"], 1, data["BB_Signal"]) # buy signal

data["BB_Signal"] = np.where(data["Close"] > data["Medium Upper Band"], -1, data["BB_Signal"]) # sell signal



In [70]:
test = data[data["BB_Signal"] !=0]

In [71]:
fig.add_trace(go.Scatter(x=data.index[data["BB_Signal"]==1], y = data["Close"][data["BB_Signal"]==1], 
                         mode = "markers", marker_color = "darkgreen", marker_symbol = "arrow-up", marker_size = 10,
                        name = "Buy Signal")) #buy signal
fig.add_trace(go.Scatter(x=data.index[data["BB_Signal"]==-1], y = data["Close"][data["BB_Signal"]==-1], 
                         mode = "markers", marker_color = "darkred", marker_symbol = "arrow-down", marker_size = 10,
                        name = "Sell Signal")) #sell signal

In [28]:
def get_returns(data):
    entrie_prices = []
    exit_prices = []
    entrie_dates = []
    exit_dates = []
    in_market = False
    for i in range(data.shape[0]):
        if (in_market == False) and (data.iloc[i]["BB_Signal"] == 1):
            in_market = True
            entrie_prices.append((data.iloc[i]["Close"]))
            entrie_dates.append(data.iloc[i].name)
        if in_market == True and (data.iloc[i]["BB_Signal"] == -1):
            in_market = False
            exit_prices.append((data.iloc[i]["Close"]))
            exit_dates.append(data.iloc[i].name)
    if len(entrie_prices) > len(exit_prices):
        exit_prices.append(data.iloc[-1]["Close"])
        exit_dates.append(data.iloc[-1].name)
    return sum(exit_prices)-sum(entrie_prices),entrie_dates, exit_dates

returns, entries, exits = get_returns(data)

In [29]:
entries, exits, returns

([Timestamp('2022-04-21 00:00:00')],
 [Timestamp('2022-07-27 00:00:00')],
 0.6800003051757812)

In [ ]:
#Volume based indacior of setntemt 

In [ ]:
data = pd.read_csv("drive/MyDrive/Grad School/Ukraine War Project/Data/Twitter_Data/samp_twitter_sentiment-2.csv", usecols = ['tweet_id', 
                                                                                                                              'text', 
                                                                                                                              'created_at',
                                                                                                                              'retweets', 
                                                                                                                              'favorites', 
                                                                                                                              'Search_Key-Words', 
                                                                                                                              'sentiment'])

data.dtypes

tweet_id              int64
text                 object
created_at           object
retweets            float64
favorites           float64
Search_Key-Words     object
sentiment            object
dtype: object

In [ ]:
data.head()

,tweet_id,text,created_at,retweets,favorites,Search_Key-Words,sentiment
0,1521912056265465861,Russia daily bombs the territory of Ukraine in...,2022-05-04 17:58:04+00:00,1.0,1.0,ukraine,"(-1, 0.9415012)"
1,1520152221761126400,I swear to fucking god mouthbreathers on this ...,2022-04-29 21:25:07+00:00,0.0,1.0,ukraine+energy,"(-1, 0.94863486)"
2,1521596935584600064,The complex effort to hold Vladimir Putin acco...,2022-05-03 21:05:54+00:00,0.0,1.0,putin+war,"(0, 0.53680235)"
3,1521361803476828160,What You Missed This Week in EVs and Clean Ene...,2022-05-03 05:31:34+00:00,0.0,0.0,stock+energy,"(0, 0.8392952)"
4,1519224592497352704,"Russia's state-run energy company, Gazprom, cu...",2022-04-27 07:59:03+00:00,0.0,0.0,natural+gas,"(0, 0.55743986)"


In [ ]:
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv("drive/MyDrive/Grad School/Ukraine War Project/Data/Twitter_Data/samp_twitter_sentiment-2.csv", usecols = ['tweet_id', 
                                                                                                                              'text', 
                                                                                                                              'created_at',
                                                                                                                              'retweets', 
                                                                                                                              'favorites', 
                                                                                                                              'Search_Key-Words', 
                                                                                                                              'sentiment'])

# Separate the sentiment and confidence columns
df['confidence'] = df['sentiment'].apply(lambda x: x[1])
df['sentiment'] = df['sentiment'].apply(lambda x: x[0])


# Convert the created_at column to datetime
df['created_at'] = pd.to_datetime(df['created_at'], utc=True)

# Group the data by date and sentiment, and count the number of tweets
df_grouped = df.groupby([pd.Grouper(key='created_at', freq='D'), 'sentiment']).size().reset_index(name='count')

# Create a color map for the sentiment values
color_map = {-1: 'red', 0: 'yellow', 1: 'blue'}

# Create a scatter plot of the tweet volumes
fig = px.scatter(df_grouped, x='created_at', y='count', color='sentiment', color_discrete_map=color_map,
                 opacity=0.5, hover_data={'count': ':,d'})
fig.update_layout(title='Daily Tweet Volume by Sentiment', xaxis_title='Date', yaxis_title='Tweet Volume')
fig.show()
